In [1]:
import pandas as pd

In [2]:
file_path = "./서울시 안전표지 관련 정보.csv"
df = pd.read_csv(file_path, encoding='euc-kr')

C:\Users\yeeun\AppData\Local\Temp\ipykernel_2212\745944131.py:2: DtypeWarning: Columns (0,1,10,18) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, encoding='euc-kr')


In [3]:
df.head()

설치방식 상태 (공통) 표지코드    비고           X좌표           Y좌표       설치일       교체일  \
0  003     001  002        197430.82666  551261.04855  20130503  20130503   
1  999     001  003  T형설치  198212.57833  553261.42560                       
2  003     001             200019.71305  554940.85846                       
3  003     001  003        199996.21210  554853.25479  20141109  20141109   
4  003     001  002        199994.21744  554963.77265  20141109  20141109   

          표지관리번호         지주관리번호 각도 (공통)  작업구분 (공통)  표출구분 (공통)  신규정규화ID  \
0  04-0000292950  02-0000145469       0          1          2  3259401   
1  04-0000294826  02-0000178517       0          6          2  3373001   
2  04-0000296447  02-0000149786       0          1          1  4306633   
3  04-0000129832  02-0000074717       0          1          2            
4  04-0000296302  02-0000149781     180          6          2  4306641   

     이력ID         공사관리번호   표지관리번호.1 표지인덱스 위치정보 공사형태  
0  313963  2013-0208-003  04-292950   322       001  
1  315839  2013-0208-001  04-294826   428       001  
2  317460                 04-296447   324       001  
3   99842  2014-0208-031  04-129832   751    9  001  
4  317315  2014-0208-031  04-296302   219       001

In [4]:
df_safety_sign = df[['비고', 'X좌표', 'Y좌표']]
df_safety_sign

비고           X좌표           Y좌표
0             197430.82666  551261.04855
1       T형설치  198212.57833  553261.42560
2             200019.71305  554940.85846
3             199996.21210  554853.25479
4             199994.21744  554963.77265
...      ...           ...           ...
349995        200206.09163  549129.25680
349996        199043.00000  551338.78125
349997        198998.59314  551529.46674
349998        198959.17804  551573.54977
349999        198959.73857  551572.89364

[350000 rows x 3 columns]

In [5]:
df_safety_sign.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 350000 entries, 0 to 349999
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   비고      350000 non-null  object 
 1   X좌표     350000 non-null  float64
 2   Y좌표     350000 non-null  float64
dtypes: float64(2), object(1)
memory usage: 8.0+ MB


복지시설 300m 반경 개수 세기

In [6]:
import geopandas as gpd
from shapely.geometry import Point


In [7]:
df_welfare = pd.read_csv("./geocode_welfare_transformed.csv")

In [8]:
df_welfare.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 677 entries, 0 to 676
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   시군구명         677 non-null    object 
 1   시설명          677 non-null    object 
 2   시설주소         677 non-null    object 
 3   시설종류명(시설유형)  677 non-null    object 
 4   정원(수용인원)     627 non-null    float64
 5   현인원          628 non-null    float64
 6   X            677 non-null    float64
 7   Y            677 non-null    float64
dtypes: float64(4), object(4)
memory usage: 42.4+ KB


In [9]:
# 각 데이터프레임에서 필요한 열만 선택
df_welfare_coords = df_welfare[['X', 'Y']].dropna().reset_index(drop=True)
len_df_welfare_coords = len(df_welfare_coords)
print(len_df_welfare_coords)
df_safety_sign_coords = df_safety_sign[['X좌표', 'Y좌표']].dropna().reset_index(drop=True)
len_df_safety_sign_coords = len(df_safety_sign_coords)


677


In [10]:
# GeoDataFrame 생성
gdf_welfare = gpd.GeoDataFrame(df_welfare_coords, geometry=gpd.points_from_xy(df_welfare_coords['X'], df_welfare_coords['Y']))
gdf_safety_sign = gpd.GeoDataFrame(df_safety_sign_coords, geometry=gpd.points_from_xy(df_safety_sign_coords['X좌표'], df_safety_sign_coords['Y좌표']))


In [11]:
# 300m 반경 내의 좌표 찾기
buffer_radius = 300  # 300m 반경
result = []

for index, welfare_point in gdf_welfare.iterrows():
    # 좌표 중심에서 300m 반경의 영역을 생성
    buffer_area = welfare_point.geometry.buffer(buffer_radius)
    
    # gdf_safety_sign에서 buffer_area와 교차하는 좌표 찾기
    intersected_signs = gdf_safety_sign[gdf_safety_sign.geometry.intersects(buffer_area)]
    
    # 결과 저장
    result.append({'welfare_coords': welfare_point[['X', 'Y']].tolist(),
                   'num_intersected_signs': len(intersected_signs)})

# 결과 출력
for res in result:
    print(f"Welfare Coords: {res['welfare_coords']}, Num Intersected Safety Signs: {res['num_intersected_signs']}")

Welfare Coords: [208651.96906081215, 545105.8098333739], Num Intersected Safety Signs: 294
Welfare Coords: [208317.7424097984, 545187.4109392836], Num Intersected Safety Signs: 318
Welfare Coords: [192899.81259540585, 543756.2357893371], Num Intersected Safety Signs: 10
Welfare Coords: [203745.06404180807, 545548.855806653], Num Intersected Safety Signs: 278
Welfare Coords: [191966.14165143383, 551578.632818147], Num Intersected Safety Signs: 196
Welfare Coords: [207088.52492535577, 557709.9623762729], Num Intersected Safety Signs: 252
Welfare Coords: [208478.481821068, 556073.310077563], Num Intersected Safety Signs: 272
Welfare Coords: [190316.88151611897, 542464.1983665512], Num Intersected Safety Signs: 237
Welfare Coords: [210568.9483171037, 541186.0524238386], Num Intersected Safety Signs: 76
Welfare Coords: [205015.56204986497, 564547.0587318793], Num Intersected Safety Signs: 167
Welfare Coords: [206471.52777337804, 560533.6658801083], Num Intersected Safety Signs: 225
Welfare 